In [1]:
import pandas as pd
import requests
from math import ceil
import json

from cyber_sdk.core.distribution.msgs import MsgWithdrawDelegationReward
from cyber_sdk.core.bech32 import AccAddress, ValAddress
from cyber_sdk.client.lcd import LCDClient

from src.tx import get_tx
from config import DELEGATOR_ADDRESS, LCD_API

MSGS_IN_TX = 16
CLAIMED_REWARDS = 48
INITIAL_SEQUENCE = 236

cyber = LCDClient("https://lcd.bostrom.cybernode.ai/", "bostrom")

In [2]:
rewards_raw = \
    requests.get(LCD_API + f'/cosmos/distribution/v1beta1/delegators/{DELEGATOR_ADDRESS}/rewards?pagination.limit=1000').json()
rewards_df = pd.DataFrame(rewards_raw['rewards'])
rewards_df.loc[:, 'reward_amount'] = rewards_df['reward'].map(lambda x: float(x[0]['amount']) if len(x) > 0 and x[0]['denom'] == 'boot' else 0)

top_rewards_df = rewards_df.sort_values('reward_amount', ascending=False).head(CLAIMED_REWARDS)
validators_list = top_rewards_df.validator_address.to_list()

print(f'Total claimable reward {int(sum(rewards_df.reward_amount)):>18,}')
print(f'Total top{CLAIMED_REWARDS} reward     {int(sum(top_rewards_df.reward_amount)):>18,}')

Total claimable reward  1,311,567,041,445
Total top48 reward      1,155,883,354,524


In [3]:
counter = ceil(len(validators_list) / MSGS_IN_TX)
for i in range(counter):
    msgs = [
        MsgWithdrawDelegationReward(
            delegator_address=AccAddress(DELEGATOR_ADDRESS),
            validator_address=ValAddress(_validator_address)).to_data()
        for _validator_address in validators_list[i * MSGS_IN_TX: min((i + 1) * MSGS_IN_TX, len(validators_list))]]
    tx = get_tx(
        messages=msgs,
        memo='',
        gas_limit=5_000_000)
    with open(f"txs/{INITIAL_SEQUENCE + i}_unsigned_reward_{i}.json", "w") as fp:
        json.dump(tx, fp, indent=4)